# Семинар - Сверточные нейронные сети (Convolution neural network)

In [ ]:
import random
import numpy as np
import pandas as pd

from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import accuracy_score

## Загрузим данные

In [ ]:
import torch
import torchvision

from torchvision import datasets,transforms

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [ ]:
trainset = datasets.CIFAR10('~/.pytorch/cifar_data/', train = True, download = True)
testset = datasets.CIFAR10('~/.pytorch/cifar_data/', train = False, download = True)

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='~/.pytorch/cifar_data/', train=True,
                                        download=True, transform=transform)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=200,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='~/.pytorch/cifar_data/', train=False,
                                       download=True, transform=transform)

testloader = torch.utils.data.DataLoader(testset, batch_size=200,
                                         shuffle=False, num_workers=2)

In [ ]:
class_names = trainset.classes
num_classes = len(trainset.classes)

In [ ]:
# Сгенерируем случаные примеры для каждого класса
fig = plt.figure(figsize=(12,5))
for i in range(num_classes):
    ax = fig.add_subplot(2, 5, 1 + i, xticks=[], yticks=[])
    idx = np.where(np.array(trainset.targets)==i)[0]
    
    features_idx = trainset.data[idx,::]
    
    img_num = np.random.randint(features_idx.shape[0])
    im = np.transpose(features_idx[img_num,::],(0,1,2))
    ax.set_title(class_names[i])
    plt.imshow(im)
plt.show()

In [ ]:
trainset.data.shape, testset.data.shape

## Парочка метрик

In [ ]:
def accuracy_score(model, testloader):
    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: %d %%' % (
        100 * correct / total))

def accuracy_score_by_class(model, testloader):  
    class_correct = list(0. for i in range(10))
    class_total = list(0. for i in range(10))
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            c = (predicted == labels).squeeze()
            for i in range(4):
                label = labels[i]
                class_correct[label] += c[i].item()
                class_total[label] += 1


    for i in range(10):
        print('Accuracy of %5s : %2d %%' % (
            class_names[i], 100 * class_correct[i] / class_total[i]))

## Сверточные нейронные сети

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return F.softmax(x, dim = 1)

net = Net()

#### Обучим модель! 

In [ ]:
learning_rate = 0.001
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=learning_rate)

for i, epoch in enumerate(range(30)):  # loop over the dataset multiple times

    for j, data in (enumerate(trainloader, 0)):
        
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        if j % 20 == 19:    # print every 20 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, j + 1, loss.item()))
    
print('Finished Training')

In [ ]:
accuracy_score(net, testloader)
print('='*50)
accuracy_score_by_class(net, testloader)

## Сохранение и загрузка модели

#### Сохранение и... 

In [ ]:
# torch.save(net.state_dict(), './net.pth')

#### ... загрузка

In [ ]:
loaded_model = Net()
loaded_model.load_state_dict(torch.load('./net.pth'))

In [ ]:
loaded_model.eval()

In [ ]:
accuracy_score(loaded_model, testloader)
print('='*50)
accuracy_score_by_class(loaded_model, testloader)

## Как нейронка видит котиков

In [ ]:
idx = np.random.choice(np.where(np.array(testset.targets)==3)[0])

im = np.transpose(testset.data[idx],(0,1,2))

subset_indices = [idx] # select your indices here as a list
subset = torch.utils.data.Subset(testset, subset_indices)
testloader_subset = torch.utils.data.DataLoader(subset, batch_size=10, num_workers=0, shuffle=False)

with torch.no_grad():
    for data in testloader_subset:
        image, label = data
        output = loaded_model(image)

print(f'True class: {class_names[testset.targets[idx]]}')
print(f'Predicted: {class_names[output.argmax().item()]} with prob: {np.round(output.max().item(), decimals=4)}')
plt.title('Image')
plt.imshow(im)
plt.show()

In [ ]:
fig = plt.figure(figsize=(12, 7))

masks = loaded_model.conv1(image)[0]
for i, mask in enumerate(masks):
    plt.subplot(2, 3, i+1)
    plt.imshow(mask.detach().numpy())
plt.show()

In [ ]:
fig = plt.figure(figsize=(16, 4))

masks = loaded_model.conv2(loaded_model.conv1(image))[0]

for i, mask in enumerate(masks):
    plt.subplot(2, 8, i+1)
    plt.imshow(mask.detach().numpy())
plt.show()



## Предобученные сети

Подробнее в [документации](https://pytorch.org/docs/stable/torchvision/models.html) 

In [ ]:
import torchvision.models as models
resnet18 = models.resnet18(pretrained=True)

In [ ]:
# Freeze model weights
for param in resnet18.parameters():
    param.requires_grad = False

In [ ]:
# Add on classifier
resnet18.fc = torch.nn.Sequential(
                      torch.nn.Linear(512, 10))
for param in resnet18.fc.parameters():
    param.requires_grad = True

In [ ]:
# specify loss function (categorical cross-entropy)
criterion = torch.nn.CrossEntropyLoss()

# specify optimizer (stochastic gradient descent) and learning rate
optimizer = torch.optim.Adam(resnet18.fc.parameters(), lr=0.00001)

In [ ]:
for i, epoch in enumerate(range(50)):  # loop over the dataset multiple times

    for j, data in (enumerate(trainloader, 0)):
        
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = resnet18(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        if j % 100 == 99:    # print every 20 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, j + 1, loss.item()))
    
print('Finished Training')

In [ ]:
accuracy_score(resnet18, testloader)
print('='*50)
accuracy_score_by_class(resnet18, testloader)

In [ ]:
# torch.save(resnet18.state_dict(), './resnet.pth')

In [ ]:
loaded_resnet = resnet18
loaded_resnet.load_state_dict(torch.load('./resnet.pth'))

In [ ]:
accuracy_score(loaded_resnet, testloader)
print('='*50)
accuracy_score_by_class(loaded_resnet, testloader)